# Notebook exercise: Converting a JONSWAP spectrum into a time series

In this interactive notebook you will try to convert a JONSWAP spectrum into a time series $\eta(t)$. For this, you will use the `MHKiT` package. 

Click {fa}`rocket` --> {guilabel}`Live Code` on the top right corner of this screen and then wait until all cells are executed.

In [10]:
!pip install mhkit
import numpy as np
import matplotlib.pyplot as plt

class Cyl:
    def __init__(self, x0, z0, R, dth):
        thMat = np.arange(0, 360, dth) #deg
        cx = x0 + R*np.cos( np.deg2rad(thMat) )
        cz = z0 + R*np.sin( np.deg2rad(thMat) )
        self.R = R
        self.D = 2*R
        self.x0 = x0
        self.z0 = z0
        self.cx = cx
        self.cz = cz
        self.NodeC = [ [x, z] for x,z in zip (cx, cz) ]
        self.nNode = len(self.NodeC)
        self.Ele = [ [n1, n2] for n1,n2 in 
                    zip( range(0,self.nNode-1), range(1,self.nNode)) ]
        self.Ele.append([self.nNode-1, 0])
        self.nEle = len(self.Ele)
        self.P = np.zeros(self.nNode)
        
    def plotEle(self):
        for iEle in range(0,self.nEle):
            n1, n2 = self.Ele[iEle]
            n1 = int(round(n1))
            n2 = int(round(n2))
            plt.plot( [self.NodeC[n1][0], self.NodeC[n2][0]],
                     [self.NodeC[n1][1], self.NodeC[n2][1]],
                     lw=3, color='k')

### 1. Generate a wave spectrum using MHKiT

Hint: use the `mhkit.wave.resource.jonswap_spectrum()` package

In [ ]:
def get_spec(w, Tp, Hs):
    q = # your code here
    return q.to_numpy()/2/ np.pi

SyntaxError: invalid syntax (3049571372.py, line 2)

In [ ]:
Hm0 = 8.0  # Significant wave height [m]
Tp = 10.0  # Peak period [s]

omega = # your code here
S = # your code here

plt.plot(omega, S)
plt.show()

### 2. Get the amplitude spectrum for the power spectral density function

Definition of single-sided power spectral density function is given by
$$
\sum_f^{f+\Delta f} \frac{1}{2} a_n^2 = S_n(f) \Delta f \quad
$$
PSDF has a unit of $m^2 \cdot s$

The single-sided amplitude spectrum is given by 
$$
a_n(f) = \sqrt{2 S_n(f) \Delta f}
$$

Here `S.args`$= \omega$ and `S.data`= Spectral density value

In [7]:
def getAmpSPEC(omega, E, iseed=None):
    df = # your code here  # Frequency band width 
    A = # your code here
    n_omega = len(omega)

    # Seed for random phase    
    if iseed is not None:
        try:
            np.random.set_state(iseed)
        except (KeyError, TypeError):
            np.random.seed(iseed)
    ph = np.random.rand(n_omega) * 2 * np.pi - np.pi
    return A[2:], omega[2:], ph[2:]

SyntaxError: invalid syntax (4256513465.py, line 2)

In [5]:
A, omega, ph = getAmpSpec(omega, S, iseed=123)
# A, w, ph = getAmpSpec(spec, ns//2+1, iseed=123)

fig, ax = plt.subplots(1,1)
plt.plot(omega, A, color='r')
plt.title('Statistical amplitude spectrum')
plt.xlabel('$\omega$ (rad/s)')
plt.ylabel('$\eta_0$ (m)')
plt.grid('on')
plt.show()

<>:7: SyntaxWarning: invalid escape sequence '\o'
<>:8: SyntaxWarning: invalid escape sequence '\e'
<>:7: SyntaxWarning: invalid escape sequence '\o'
<>:8: SyntaxWarning: invalid escape sequence '\e'
/var/folders/4r/myg3jzhx1yd72kbqs5r4rv2w0000gn/T/ipykernel_66684/782426865.py:7: SyntaxWarning: invalid escape sequence '\o'
  plt.xlabel('$\omega$ (rad/s)')
/var/folders/4r/myg3jzhx1yd72kbqs5r4rv2w0000gn/T/ipykernel_66684/782426865.py:8: SyntaxWarning: invalid escape sequence '\e'
  plt.ylabel('$\eta_0$ (m)')
/var/folders/4r/myg3jzhx1yd72kbqs5r4rv2w0000gn/T/ipykernel_66684/782426865.py:7: SyntaxWarning: invalid escape sequence '\o'
  plt.xlabel('$\omega$ (rad/s)')
/var/folders/4r/myg3jzhx1yd72kbqs5r4rv2w0000gn/T/ipykernel_66684/782426865.py:8: SyntaxWarning: invalid escape sequence '\e'
  plt.ylabel('$\eta_0$ (m)')


NameError: name 'getAmpSpec' is not defined

### 3. Calculate the elevation profile

We use `LinearWave2D` class from `LinearWave` package for defining the linear wave and the associated particle velocity, acceleration and pressure fields.

The linear wave theory only calculate the pressure and velocities for $z<=0$. <br> In order to calculate the values for $z>0$ we use

- For Pressure: Taylor series expansion, limited to first order
- For Velocities: Wheeler stretching [link](https://www.orcina.com/webhelp/OrcaFlex/Content/html/Waves,Kinematicstretching.htm#:~:text=Wheeler%20stretching,at%20the%20instantaneous%20water%20surface.)

We initiate `LineaerWave` object using amplitudes from JONSWAP spectrum and random phase. This will be used for obtaining the elevation, velocity and acceleration time-series for calculating the forces.

We have two classes

- `LinearWave2D`: Class to define wave in shallow water
    - `wv = LinearWave2D(rhoW, g, d, T, H)`
    - `rhoW`: Density of water
    - `g`: acceleration due to gravity
    - `d`: still-water depth
    - `T`: Wave time-period
    - `H`: Wave-height

- `LinearWaveDeep2D`: Class to define wave in deep water
    - `wv = LinearWave2D(rhoW, g, T, H)`
    - `rhoW`: Density of water
    - `g`: acceleration due to gravity
    - `T`: Wave time-period
    - `H`: Wave-height


Location in 2D space is defined as `(x,z)`
The time instant is `t`

- `wv.waveElevation(t,x)` : Wave elevation at a given x
- `wv.pressureTot(t,x,z)` : Total pressure (static + dynamic pressure)
- `wv.pressureDyn(t,x,z)` : Dynamic pressure
- `wv.particleVelPoi(t,x,z)` : Wave particle vel at (x,z) at time-instant t
- `wv.particleAccPoi(t,x,z)` : Wave particle acceleration at (x,z) at time-instant t
- `wv.particleVelMax(t,x,z)` : Maximum of wave particle vel at location (x,z)
- `wv.particleAccMax(t,x,z)` : Maximum of wave particle acceleration at location (x,z)

Here `wv` can be an object of `LinearWave2D` or `LinearWaveDeep2D`

In [ ]:
from LinearWave import LinearWave2D
from LinearWave import LinearWaveDeep2D

# Wave(g, d, T, H, phi=0, x0=0)
d = 187 #m (still-water depth)
rhoW = 1025
g = 9.81
cyl1 = Cyl(10, -20, 5, 0.1)

wvAll = [ LinearWaveDeep2D(rhoW, g, 2*np.pi/wi, 2*Ai, phi, msg=False) for wi,Ai,phi in zip(w,A,ph) ]

ModuleNotFoundError: No module named 'LinearWave'

In [9]:
def spec2ts(wvListIn, x0, z0, t):
    nt = len(t)
    et_t = np.zeros(nt) # wave elevation time series
    vx_t = np.zeros(nt) # horizontal particle velocity time series
    vz_t = np.zeros(nt) # vertical particle velocity time series
    vm_t = np.zeros(nt) # Velocity magnitude time series
    ax_t = np.zeros(nt) # Horizontal particle acceleration time series
    az_t = np.zeros(nt) # Vertical particle acceleration time series
    
    for i, ti in enumerate(t):
        et_t[i] = sum([wv.waveElevation(ti, x0) for wv in wvListIn])
        vel = np.array([wv.particleVelPoi(ti, x0, z0) for wv in wvListIn])
        vx_t[i] = np.sum(vel[:, 0])
        vz_t[i] = np.sum(vel[:, 1])
        vm_t[i] = np.sqrt( vx_t[i]**2 + vz_t[i]**2 )
        acc = np.array([wv.particleAccPoi(ti, x0, z0) for wv in wvListIn])
        ax_t[i] = np.sum(acc[:, 0])
        az_t[i] = np.sum(acc[:, 1])
    
    return et_t, vx_t, vz_t, vm_t, ax_t, az_t

In [ ]:
fig, ax = plt.subplots(1,1)
plt.plot(t_t, et_t, color='r')
plt.title('Wave Elevation')
plt.xlabel('t (s)')
plt.ylabel('$\eta$ (m)')
plt.grid('on')

fig, ax = plt.subplots(1,1)
plt.plot(t_t, vx_t, color='r', label='vx')
plt.plot(t_t, vz_t, color='b', label='vz')
plt.plot(t_t, vm_t, color='g', label='vMag')
plt.title('Vel')
plt.xlabel('t (s)')
plt.ylabel('v (m/s)')
plt.grid('on')
plt.legend()

fig, ax = plt.subplots(1,1)
plt.plot(t_t, ax_t, color='r', label='ax')
plt.plot(t_t, az_t, color='b', label='az')
plt.title('Acc')
plt.xlabel('t (s)')
plt.ylabel('$\dot{v}$ (m/s2)')
plt.grid('on')
plt.legend()

plt.show()